### Dataset Structure

```
dataset/
├── train/
│   ├── plastic/
│   ├── paper/
│   ├── glass/
│   └── ...
├── val/
│   ├── plastic/
│   ├── paper/
│   └── ...
```

Source: Kaggle Garbage Classification<br></br>
Link: https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification?resource=download

##### Import Required Libraries

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

##### Data Processing

In [6]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = datasets.ImageFolder(r'C:\Users\chang\OneDrive\文件\ELLIE\Garbage Classification\dataset\train', transform=transform)
val_dataset = datasets.ImageFolder(r'C:\Users\chang\OneDrive\文件\ELLIE\Garbage Classification\dataset\val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


##### Define a Simple CNN

In [7]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)

##### Train the model

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN(num_classes=len(train_dataset.classes)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1, Loss: 1.3791
Epoch 2, Loss: 0.9540
Epoch 3, Loss: 0.7763
Epoch 4, Loss: 0.5667
Epoch 5, Loss: 0.4157
Epoch 6, Loss: 0.2661
Epoch 7, Loss: 0.1714
Epoch 8, Loss: 0.1110
Epoch 9, Loss: 0.0624
Epoch 10, Loss: 0.0556


##### Evaluate the model

In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')


Validation Accuracy: 61.81%


##### Save the model

In [10]:
torch.save(model.state_dict(), 'garbage_classifier.pth')

##### Predict

###### Load the model

In [11]:
model = SimpleCNN(num_classes=6)  # Use same model class
model.load_state_dict(torch.load('garbage_classifier.pth'))
model.eval()  # Set to evaluation mode


SimpleCNN(
  (net): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=32768, out_features=128, bias=True)
    (8): ReLU()
    (9): Linear(in_features=128, out_features=6, bias=True)
  )
)

###### Load a single image

In [12]:
from PIL import Image
from torchvision import transforms

# Define same transform as used in training
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load your image
img = Image.open('example.jpg').convert('RGB')  # RGB is important
img_tensor = transform(img)
img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension: shape becomes [1, 3, 128, 128]


###### Predict

In [15]:
with torch.no_grad():
    outputs = model(img_tensor)  # Forward pass
    _, predicted = torch.max(outputs, 1)  # Get index of highest score

print(f"Predicted class: {train_dataset.classes[predicted.item()]}")


Predicted class: cardboard
